In [ ]:
from io import StringIO
from PIL import Image
import glob
import os
import sys
import numpy as np
import pandas as pd 
import time
import datetime
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
validation_csv_path = 'Annotation/test-annotations-bbox.csv'
path = 'testDataset/'
image_path = "testDataset/*"
training_path = 'Validation Dataset/training_set/'
test_path =  'Validation Dataset/test_set/'

In [3]:
files_path=glob.glob(image_path)
files_names =[]
for n in range(len(files_path)):
    filename_w_ext = os.path.basename(files_path[n])
    filename, file_extension = os.path.splitext(filename_w_ext)
    files_names.append(filename)

df_files_names=pd.DataFrame(files_names,columns=['ImageID'])
df_files_names = df_files_names.drop_duplicates()
len(df_files_names)

118057

In [4]:
validation_df = pd.read_csv(validation_csv_path)
validation_df.head()

,ImageID,Source,LabelName,Confidence,XMin,XMax,YMin,YMax,IsOccluded,IsTruncated,IsGroupOf,IsDepiction,IsInside
0,000026e7ee790996,freeform,/m/07j7r,1,0.071905,0.145346,0.206591,0.391306,0,1,1,0,0
1,000026e7ee790996,freeform,/m/07j7r,1,0.439756,0.572466,0.264153,0.435122,0,1,1,0,0
2,000026e7ee790996,freeform,/m/07j7r,1,0.668455,1.000000,0.000000,0.552825,0,1,1,0,0
3,000062a39995e348,freeform,/m/015p6,1,0.205719,0.849912,0.154144,1.000000,0,0,0,0,0
4,000062a39995e348,freeform,/m/05s2s,1,0.137133,0.377634,0.000000,0.884185,1,1,0,0,0


In [ ]:
LabelName = validation_df['LabelName']
LabelName = LabelName.drop_duplicates()
x = LabelName.str.split('/m/')
for i in range(len(x)):
    folder_name = x.iloc[i][1]
    if not os.path.exists(training_path+folder_name):
        os.makedirs(training_path+folder_name)
    if not os.path.exists(test_path+folder_name):
        os.makedirs(test_path+folder_name)    

In [5]:
df_noduplicate_image = validation_df['ImageID']
df_noduplicate_image = df_noduplicate_image.drop_duplicates()
df_noduplicate_image=df_noduplicate_image.astype(str)
df_files_names = df_files_names.astype(str)
len(df_noduplicate_image), len(df_files_names)

(108159, 118057)

In [8]:
# Shuffle Data Frame
time_frame=[]
total_start_time = time.time()
# validation_df = validation_df.sample(frac=1).reset_index(drop=True)
df_size = len(validation_df)
# i=0
for i in range(df_size):
    found_flag = df_files_names[df_files_names['ImageID'].str.contains(df_noduplicate_image.iloc[i])].any().iloc[0]
    if found_flag:
        start = time.time()
        LabelName = validation_df['LabelName'].iloc[i]
        LabelName = LabelName.split('/m/')
        folder_name = LabelName[1]
        image_path = path+validation_df['ImageID'].iloc[i]+'.jpg'
        image = Image.open(image_path)
        W, H = image.size
        box = (validation_df['XMin'].iloc[i]* W, validation_df['YMin'].iloc[i]*H, validation_df['XMax'].iloc[i]*W, validation_df['YMax'].iloc[i]*H)
        cropped_img = image.crop(box)
        w, h = cropped_img.size
        if w>44 and h>44:
            split_ratio = i/df_size
            if split_ratio<0.8:
                folder_path = training_path+folder_name+'/'
            else:
                folder_path = test_path+folder_name+'/'
            cropped_img = cropped_img.resize((128,128))
            cropped_img.save(folder_path+validation_df['ImageID'].iloc[i]+'_'+folder_name+'_'+str(i)+'.jpg')
            time.sleep(0.001)
    #         if i%100==0:
        end=time.time()-start
        ET = end * (df_size-i)
        sys.stdout.write("\r" +'ET= '+ str(datetime.timedelta(seconds=ET)) + ' ... '+str(np.round(i/df_size,4))+' % ' )    
total_end_time = time.time() - total_start_time 
print('Total End Time= ', str(datetime.timedelta(total_end_time)))

ET= 4:48:06.890841 ... 0.1628 %  1618 % % 

IndexError: single positional indexer is out-of-bounds